<a href="https://colab.research.google.com/github/duocclases/SCA-202101-MineriaDatos-02-TEAM01/blob/main/lab2_grupo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>